In [1]:
import tensorflow as tf
from keras import layers
from keras import preprocessing
import word2vec as w2v
import dataProsessing as dp
import numpy as np
from tensorboard.plugins import projector
import os
import io
import json

In [2]:

#run this once and only once
os.chdir("../")
print(os.getcwd())

d:\desktop\MAT_180_ML_Projects\corpus-of-chords


In [16]:
datasetName = "Jsb16thSeparated(t_60_rr_re)"

corpus = dp.loadDataset("Data\\Corpi\\" + datasetName + "\\" + datasetName +".json")
vocab = dataset = json.load(open("Data\\Corpi\\" + datasetName + "\\" + datasetName +"_vocab.json"))
inv_vocab = vocab["inv_vocab"]
vocab = vocab["vocab"]



In [17]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
embedding_dim = 128
negative_samples = 15
log_dir = "logs\embeddings"
model_name =  "d_" + str(embedding_dim) + " n_" + str(negative_samples)

vectorizedSequences = dp.vectorizeDataset(corpus,vocab)
targets,contexts,labels = [],[],[]
for s in vectorizedSequences:
    tmp = w2v.generateTrainingData(s,vocab_size=len(vocab),window_size=1,negative_samples=negative_samples)
    targets += tmp[0]
    contexts += tmp[1]
    labels += tmp[2]
    
targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

dataset = tf.data.Dataset.from_tensor_slices(((targets,contexts),labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


word2vec = w2v.Word2Vec(len(vocab), embedding_dim,4)
word2vec.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

weights = tf.Variable(word2vec.get_layer('w2v_embedding').get_weights()[0])


checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join("data\\corpi\\" + datasetName + "\\tensorboard data\\" + model_name, "embedding.ckpt"))






Epoch 1/20
64/64 [==============================] - 4s 47ms/step - loss: 2.7594 - accuracy: 0.2803
Epoch 2/20
64/64 [==============================] - 3s 47ms/step - loss: 2.6617 - accuracy: 0.7898
Epoch 3/20
64/64 [==============================] - 3s 47ms/step - loss: 2.3866 - accuracy: 0.8750
Epoch 4/20
64/64 [==============================] - 3s 48ms/step - loss: 1.9862 - accuracy: 0.8477
Epoch 5/20
64/64 [==============================] - 3s 48ms/step - loss: 1.6279 - accuracy: 0.8365
Epoch 6/20
64/64 [==============================] - 3s 47ms/step - loss: 1.3547 - accuracy: 0.8447
Epoch 7/20
64/64 [==============================] - 3s 45ms/step - loss: 1.1383 - accuracy: 0.8634
Epoch 8/20
64/64 [==============================] - 3s 48ms/step - loss: 0.9588 - accuracy: 0.8855
Epoch 9/20
64/64 [==============================] - 3s 45ms/step - loss: 0.8062 - accuracy: 0.9055
Epoch 10/20
64/64 [==============================] - 3s 49ms/step - loss: 0.6771 - accuracy: 0.9240
Epoch 11/

'data\\corpi\\Jsb16thSeparated(t_60_rr_re)\\tensorboard data\\d_128 n_15\\embedding.ckpt-1'

In [18]:


weights = np.array(weights).tolist()
print(type(weights))

with open("data\\corpi\\" + datasetName + "\\weights" + model_name, "w") as outfile:
    json.dump(weights, outfile)



<class 'list'>


In [9]:
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings("data\\corpi\\" + datasetName  + "\\tensorboard data\\" + model_name, config)

In [ ]:

%tensorboard --logdir logs\embeddings
